Import Libraries

In [1]:
import pandas as pd
import numpy as np
import Saba_Scrapping as SS
from geopy.geocoders import Nominatim

Scrap Data --> Infosaba.com

In [3]:
my_chromdriver = "Z:\\HQ\\BDM\\a.zohdi\\Data Engineering\\Github\\Geocoding\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

df_industries = SS.GET_industries(my_chromdriver)
print('Industries are Done !')

urls = df_industries['URL']
df_units = SS.GET_unit_subjects(urls, my_chromdriver)
print('Units are Done !')

df_complexes = SS.GET_complexes(my_chromdriver)
print('Complexes are Done !')

urls = df_units['URL']
df_unitcomplex = SS.GET_unit_complex(urls, my_chromdriver)
print('Unit_Complex is Done !')

Industries are Done !
Units are Done !
Complexes are Done !
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بازیافت-پروپیلن
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بوتانول
There is no Complex in this link: https://infosaba.com/Unit_Subjects/Staple
There is no Complex in this link: https://infosaba.com/Unit_Subjects/اپکسی-رزین
There is no Complex in this link: https://infosaba.com/Unit_Subjects/تاسیسات-استخراج-سنگ-آهن
There is no Complex in this link: https://infosaba.com/Unit_Subjects/ورق-خودرو
There is no Complex in this link: https://infosaba.com/Unit_Subjects/مشخصات-معادن-بوکسیت
There is no Complex in this link: https://infosaba.com/Unit_Subjects/کوره‌-شمش-ریزی-آلومینیوم
There is no Complex in this link: https://infosaba.com/Unit_Subjects/اکسیداسیون
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بروشوئی-سیائیدی
There is no Complex in this link: https://infosaba.com/Unit_Subjects/خردایش
There is no Complex in thi

Get Coordinations using complex.name & OSM.name (kinda)

In [59]:
complex_coords = pd.read_excel("Z:\\HQ\\BDM\\a.zohdi\\Complex OSM.xlsx", index_col=0)
geolocator = Nominatim(user_agent="Zohdi")

# getting coordinations of complexes
complex_coords['coordination'] = np.nan
i = 0
while i < len(complex_coords):
    if not pd.isna(complex_coords.iloc[i, 3]):

        OSM_name = complex_coords.iloc[i, 3]
        location = geolocator.geocode(OSM_name)
        coordination = f"{location.latitude}, {location.longitude}"
        complex_coords.iloc[i, 5] = coordination
        i += 1

    else:

        complex_coords.iloc[i, 5] = complex_coords.iloc[i, 4]
        i += 1

# split coordination to lat-long
complex_coords[['latitude', 'longitude']] = complex_coords['coordination'].str.split(',', expand=True)

In [9]:
df_complexes = complex_coords

with pd.ExcelWriter('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx') as writer:
    df_industries.to_excel(writer, sheet_name='Industry')
    df_units.to_excel(writer, sheet_name='Unit')
    df_complexes.to_excel(writer, sheet_name='Complex')
    df_unitcomplex.to_excel(writer, sheet_name='Unit_Complex')

Slicing Unit_Complex Dataframe for each Unit

In [43]:
df_unitcomplex_joined = df_unitcomplex.merge(df_complexes, left_on='complex.name', right_on='name')
df_unitcomplex_joined.drop(columns=['unit.url', 'name', 'city', 'industry.name', 'OSM_name', 'GoogleMap_coordination'], inplace=True)

for i in df_units[df_units['industry.id'] == 4].index:
    exec("df_unitcomplex_"+str(i)+" = df_unitcomplex_joined[df_unitcomplex_joined['unit.id'] == i]")

In [54]:
# df_unitcomplex_130.astype({'capacity_value': int, 'under_construction_value': int})
# df_unitcomplex_128['capacity_value'].replace(",","")

In [55]:
from routing_osrm import table as distance_matrix

out = distance_matrix(list(df_unitcomplex_128['coordination']))

,unit.id,complex.name,capacity_value,capacity_measure,under_construction_value,under_construction_measure,coordination,latitude,longitude
1177,130,مجتمع معدنی و صنعتی گل گهر,"15,300,000",تن در سال,0,تن در سال,"29.0918765, 55.3341122",29.0918765,55.3341122
1179,130,مجتمع فولاد چادرملو,"10,500,000",تن در سال,0,تن در سال,"32.396151826229065, 53.731420339038834",32.396151826229065,53.731420339038834
1184,130,مجتمع سنگ آهن گهر زمین,"6,000,000",تن در سال,0,تن در سال,"29.11945075, 55.30752518689982",29.11945075,55.30752518689982
1187,130,مجتمع صنعتی و معدنی اپال پارسیان سنگان,"2,600,000",تن در سال,"2,400,000",تن در سال,"34.44525542987696, 60.40206921997904",34.44525542987696,60.40206921997904
1189,130,مجتمع فولاد سنگان خراسان,"5,000,000",تن در سال,0,تن در سال,"34.442600850000005, 60.38610869109604",34.442600850000005,60.38610869109604
1191,130,مجتمع فولاد زرند ایرانیان (ذوب آهن زرند کرمان),"4,000,000",تن در سال,0,تن در سال,"30.75593395, 56.66216737906281",30.75593395,56.66216737906281
1198,130,مجتمع فولاد سیرجان ایرانیان,"4,000,000",تن در سال,0,تن در سال,"29.9279746, 56.7364051265278",29.9279746,56.7364051265278
1203,130,مجتمع صنعتی ذوب آهن پاسارگاد,0,تن در سال,"3,500,000",تن در سال,"29.1888927, 52.84769215837588",29.1888927,52.84769215837588
1209,130,مجتمع سنگ آهن مرکزی ایران,"3,250,000",تن در سال,0,تن در سال,"31.695738584781417, 55.45789213236736",31.695738584781417,55.45789213236736
1212,130,مجتمع سنگ آهن سنگان,"2,600,000",تن در سال,0,تن در سال,"34.4548214, 60.41930436160712",34.4548214,60.41930436160712
